<a href="https://colab.research.google.com/github/ariahosseini/DeepML/blob/main/032_PyTorch_Proj_ThirtyTwo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# utils
import time, os, sys
import random
import numpy as np
import pandas as pd
from scipy.special import binom, gamma
from collections import OrderedDict
# sklearn
from sklearn.preprocessing import MinMaxScaler
# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
# vis
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0))
    return (cumsum[N:] - cumsum[:-N]) / float(N)

def Catalan(k):
    return binom(2*k,k)/(k+1)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using gpu: %s ' % torch.cuda.is_available())

Using gpu: False 


In [ ]:
use_gpu = torch.cuda.is_available()
def gpu(tensor, gpu=use_gpu):
    if gpu:
        return tensor.cuda()
    else:
        return tensor

In [ ]:
torch.__version__

'2.3.0+cu121'

## Gen Data

In [ ]:
seq_max_len = 20
seq_min_len = 4

In [ ]:
def all_parent(n, a, k=-1):
    global res
    if k==n-1 and sum(a) == 0:
        res.append(a.copy())
    elif k==n-1:
        pass
    else:
        k += 1
        if sum(a) > 0:
            a[k] = 1
            all_parent(n,a,k)
            a[k] = -1
            all_parent(n,a,k)
            a[k] = 0
        else:
            a[k] = 1
            all_parent(n,a,k)
            a[k] = 0

In [ ]:
def all_parent_mistake(n, a, k=-1):
    global res
    if k==n-1 and sum(a) >= -1 and sum(a) <= 1 and min(np.cumsum(a))<0:
        res.append(a.copy())
    elif sum(a) > n-k:
        pass
    elif k==n-1:
        pass
    else:
        k += 1
        if sum(a) >= -1 and k != 0:
            a[k] = 1
            all_parent_mistake(n,a,k)
            a[k] = -1
            all_parent_mistake(n,a,k)
            a[k] = 0
        else:
            a[k] = 1
            all_parent_mistake(n,a,k)
            a[k] = 0

In [ ]:
def reading_par(l, n):
    res = [0]*len(l)
    s = []
    n_plus = -1
    n_moins = n+1
    c = 0
    for i in l:
        if i == 1:
            n_plus += 1
            s.append(n_plus)
            res[c] = n_plus
            c += 1
        else:
            try:
                res[c] = n-s.pop()
            except:
                res[c] = n_moins
                n_moins += 1
            c += 1
    return res

In [ ]:
all_par = OrderedDict()
for n in range(seq_min_len,seq_max_len+1,2):
    a = [0]*n
    res = []
    all_parent(n=n,a=a,k=-1)
    all_par[n] = [reading_par(k,n) for k in res]

In [ ]:
all_par_mist = OrderedDict()
for n in range(seq_min_len,seq_max_len+1,2):
    a = [0]*n
    res = []
    all_parent_mistake(n=n,a=a,k=-1)
    all_par_mist[n] = [reading_par(k,n) for k in res]

In [ ]:
all_par[6], all_par_mist[6]

([[0, 1, 2, 4, 5, 6],
  [0, 1, 5, 2, 4, 6],
  [0, 1, 5, 6, 2, 4],
  [0, 6, 1, 2, 4, 5],
  [0, 6, 1, 5, 2, 4]],
 [[0, 1, 5, 6, 7, 2],
  [0, 6, 1, 5, 7, 2],
  [0, 6, 7, 1, 2, 4],
  [0, 6, 7, 1, 5, 2],
  [0, 6, 7, 8, 1, 2]])

In [ ]:
long_mist = {i:len(l) for (i,l) in zip(all_par_mist.keys(), all_par_mist.values())}
long_mist

{4: 1, 6: 5, 8: 20, 10: 75, 12: 275, 14: 1001, 16: 3640, 18: 13260, 20: 48450}

In [ ]:
Catalan_num = {i:len(l) for (i,l) in zip(all_par.keys(),all_par.values())}
Catalan_num

{4: 2, 6: 5, 8: 14, 10: 42, 12: 132, 14: 429, 16: 1430, 18: 4862, 20: 16796}

In [ ]:
[(2*i,Catalan(i)) for i  in range(2,int(seq_max_len/2)+1)]

[(4, 2.0),
 (6, 5.0),
 (8, 14.0),
 (10, 42.0),
 (12, 132.0),
 (14, 429.0),
 (16, 1430.0),
 (18, 4862.0),
 (20, 16796.0)]

In [ ]:
nb_symbol = 10
np.sum([Catalan(i)*int(nb_symbol/2)**i for i in range(2,int(seq_max_len/2)+1)])

174113843800.0

In [ ]:
class SequenceGenerator():
    def __init__(self, nb_symbol = 10, seq_min_len = 4, seq_max_len = 10):
        self.nb_symbol = nb_symbol
        self.seq_min_len = seq_min_len
        self.seq_max_len = seq_max_len
        self.population = [i for i in range(int(nb_symbol/2))]

    def generate_pattern(self):
        len_r = random.randint(self.seq_min_len/2,self.seq_max_len/2)
        pattern = random.choices(self.population,k=len_r)
        return pattern + pattern[::-1]

    def generate_pattern_parenthesis(self, len_r = None):
        if len_r == None:
            len_r = int(2*random.randint(self.seq_min_len/2,self.seq_max_len/2))
        pattern = np.random.choice(self.population,size=int(len_r/2),replace=True)
        ind_r = random.randint(0,Catalan_num[len_r]-1)
        res = [pattern[i] if i <= len_r/2 else self.nb_symbol-1-pattern[len_r-i] for i in all_par[len_r][ind_r]]
        return res

    def generate_parenthesis_false(self):
        len_r = int(2*random.randint(self.seq_min_len/2,self.seq_max_len/2))
        pattern = np.random.choice(self.population,size=int(len_r/2),replace=True)
        ind_r = random.randint(0,long_mist[len_r]-1)
        res = [pattern[i] if i <= len_r/2
               else  self.nb_symbol-1-pattern[len_r-i] if i<= len_r
               else self.nb_symbol-1-pattern[i-len_r] for i in all_par_mist[len_r][ind_r]]
        return res

    def generate_hard_parenthesis(self, len_r = None):
        if len_r == None:
            len_r = int(2*random.randint(self.seq_min_len/2,self.seq_max_len/2))
        pattern = np.random.choice(self.population,size=int(len_r/2),replace=True)
        ind_r = random.randint(0,Catalan_num[len_r]-1)
        res = [pattern[i] if i <= len_r/2 else self.nb_symbol-1-pattern[len_r-i] for i in all_par[len_r][ind_r]]

        if len_r == None:
            len_r = int(2*random.randint(self.seq_min_len/2,self.seq_max_len/2))
        pattern = np.random.choice(self.population,size=int(len_r/2),replace=True)
        ind_r = random.randint(0,Catalan_num[len_r]-1)
        res2 = [pattern[i] if i <= len_r/2 else self.nb_symbol-1-pattern[len_r-i] for i in all_par[len_r][ind_r]]
        return res + res2

    def generate_hard_nonparenthesis(self, len_r = None):
        if len_r == None:
            len_r = int(2*random.randint(self.seq_min_len/2,self.seq_max_len/2))
        pattern = np.random.choice(self.population,size=int(len_r/2),replace=True)
        ind_r = random.randint(0,long_mist[len_r]-1)
        res = [pattern[i] if i <= len_r/2
               else  self.nb_symbol-1-pattern[len_r-i] if i<= len_r
               else self.nb_symbol-1-pattern[i-len_r] for i in all_par_mist[len_r][ind_r]]

        if len_r == None:
            len_r = int(2*random.randint(self.seq_min_len/2,self.seq_max_len/2))
        pattern = np.random.choice(self.population,size=int(len_r/2),replace=True)
        ind_r = random.randint(0,Catalan_num[len_r]-1)
        res2 = [pattern[i] if i <= len_r/2 else self.nb_symbol-1-pattern[len_r-i] for i in all_par[len_r][ind_r]]
        return  res +[self.nb_symbol-1-pattern[0]]+ res2

    def generate_false(self):
        popu = [i for i in range(nb_symbol)]
        len = random.randint(self.seq_min_len/2,self.seq_max_len/2)
        return random.choices(popu,k=len) + random.choices(popu,k=len)

    def generate_label(self, x):
        l = int(len(x)/2)
        return 1 if x[:l] == x[:l-1:-1] else 0

    def generate_label_parenthesis(self, x):
        s = []
        label = 1
        lenx = len(x)
        for i in x:
            if s == [] and i < self.nb_symbol/2:
                s.append(i)
            elif s == [] and i >= self.nb_symbol/2:
                label = 0
                break
            elif i == self.nb_symbol-1-s[-1]:
                s.pop()
            else:
                s.append(i)
        if s != []:
            label = 0
        return label

    def one_hot(self,seq):
        one_hot_seq = []
        for s in seq:
            one_hot = [0 for _ in range(self.nb_symbol)]
            one_hot[s] = 1
            one_hot_seq.append(one_hot)
        return one_hot_seq

    def generate_input(self, len_r = None, true_parent = False, hard_false = True):
        if true_parent:
            seq = self.generate_pattern_parenthesis(len_r)
        elif bool(random.getrandbits(1)):
            seq = self.generate_pattern_parenthesis(len_r)
        else:
            if hard_false:
                seq = self.generate_parenthesis_false()
            else:
                seq = self.generate_false()
        return gpu(torch.from_numpy(np.array(self.one_hot(seq))).type(torch.FloatTensor)), gpu(torch.from_numpy(np.array([self.generate_label_parenthesis(seq)])))

    def generate_input_hard(self,true_parent = False):
        if true_parent:
            seq = self.generate_hard_parenthesis(self.seq_max_len)
        elif bool(random.getrandbits(1)):
            seq = self.generate_hard_parenthesis(self.seq_max_len)
        else:
            seq = self.generate_hard_nonparenthesis(self.seq_max_len)

        return gpu(torch.from_numpy(np.array(self.one_hot(seq))).type(torch.FloatTensor)), gpu(torch.from_numpy(np.array([self.generate_label_parenthesis(seq)])))

In [ ]:
nb_symbol = 10
generator = SequenceGenerator(nb_symbol = nb_symbol, seq_min_len = seq_min_len, seq_max_len = seq_max_len)

In [ ]:
generator.generate_pattern_parenthesis()

[3, 0, 9, 2, 4, 5, 7, 6, 1, 8]

In [ ]:
x = generator.generate_parenthesis_false()

In [ ]:
generator.generate_label_parenthesis(x)

0

In [ ]:
generator.generate_input()

(tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0

## RNN Elman Network

In [ ]:
class RecNet(nn.Module):
    def __init__(self, dim_input=10, dim_recurrent=50, dim_output=2):
        super(RecNet, self).__init__()
        self.fc_x2h = nn.Linear(dim_input, dim_recurrent)
        self.fc_h2h = nn.Linear(dim_recurrent, dim_recurrent, bias = False)
        self.fc_h2y = nn.Linear(dim_recurrent, dim_output)

    def forward(self, x):
        h = x.new_zeros(1, self.fc_h2y.weight.size(1))
        for t in range(x.size(0)):
            h = torch.relu(self.fc_x2h(x[t,:]) + self.fc_h2h(h))
        return self.fc_h2y(h)

RNN = gpu(RecNet(dim_input = nb_symbol))

In [ ]:
cross_entropy = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(RNN.parameters(),lr=learning_rate)
nb_train = 40000
loss_t = []
corrects =[]
labels = []
start = time.time()
for k in range(nb_train):
    x,l = generator.generate_input(hard_false = False)
    y = RNN(x)
    loss = cross_entropy(y,l)
    _,preds = torch.max(y.data,1)
    corrects.append(preds.item() == l.data.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_t.append(loss)
    labels.append(l.data)
print(time.time()-start)

142.6707227230072


In [ ]:
plt.plot(running_mean(loss_t, int(nb_train/100)))

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
plt.plot(running_mean(corrects,int(nb_train/100)))

In [ ]:
nb_test = 1000
corrects_test =[]
labels_test = []
for k in range(nb_test):
    x,l = generator.generate_input(len_r=seq_max_len, true_parent=True)
    y = RNN(x)
    _,preds = torch.max(y.data,1)
    corrects_test.append(preds.item() == l.data.item())
    labels_test.append(l.data)

In [ ]:
np.sum(corrects_test)/nb_test

In [ ]:
nb_test = 1000
corrects_test =[]
labels_test = []
for k in range(nb_test):
    x,l = generator.generate_input(len_r=seq_max_len, hard_false=True)
    y = RNN(x)
    _,preds = torch.max(y.data,1)
    corrects_test.append(preds.item() == l.data.item())
    labels_test.append(l.data)

In [ ]:
np.sum(corrects_test)/nb_test

In [ ]:
nb_test = 1000
correctsh_test =[]
labelsh_test = []
for k in range(nb_test):
    x,l = generator.generate_input_hard()
    y = RNN(x)
    _,preds = torch.max(y.data,1)
    correctsh_test.append(preds.item() == l.data.item())
    labelsh_test.append(l.data)

In [ ]:
np.sum(correctsh_test)/nb_test

In [ ]:
nb_test = 1000
correctsh_test =[]
labelsh_test = []
for k in range(nb_test):
    x,l = generator.generate_input_hard(true_parent=True)
    y = RNN(x)
    _,preds = torch.max(y.data,1)
    correctsh_test.append(preds.item() == l.data.item())
    labelsh_test.append(l.data)

In [ ]:
np.sum(correctsh_test)/nb_test

## RNN with Gating

In [ ]:
class RecNetGating(nn.Module):
    def __init__(self, dim_input=10, dim_recurrent=50, dim_output=2):
        super(RecNetGating, self).__init__()
        self.fc_x2h = nn.Linear(dim_input, dim_recurrent)
        self.fc_h2h = nn.Linear(dim_recurrent, dim_recurrent, bias = False)
        self.fc_x2z = nn.Linear(dim_input, dim_recurrent)
        self.fc_h2z = nn.Linear(dim_recurrent,dim_recurrent, bias = False)
        self.fc_h2y = nn.Linear(dim_recurrent, dim_output)

    def forward(self, x):
        h = x.new_zeros(1, self.fc_h2y.weight.size(1))
        for t in range(x.size(0)):
            z = torch.sigmoid(self.fc_x2z(x[t,:])+self.fc_h2z(h))
            hb = torch.relu(self.fc_x2h(x[t,:]) + self.fc_h2h(h))
            h = z * h + (1-z) * hb
        return self.fc_h2y(h)

RNNG = gpu(RecNetGating(dim_input = nb_symbol))

In [ ]:
optimizerG = torch.optim.Adam(RNNG.parameters(),lr=1e-3)
loss_tG = []
correctsG =[]
labelsG = []
start = time.time()
for k in range(nb_train):
    x,l = generator.generate_input(hard_false = False)
    y = RNNG(x)
    loss = cross_entropy(y,l)
    _,preds = torch.max(y.data,1)
    correctsG.append(preds.item() == l.data.item())
    optimizerG.zero_grad()
    loss.backward()
    optimizerG.step()
    loss_tG.append(loss)
    labelsG.append(l.item())
print(time.time() - start)

In [ ]:
plt.plot(running_mean(loss_tG, int(nb_train/50)))
plt.plot(running_mean(loss_t, int(nb_train/50)))

In [ ]:
plt.plot(running_mean(correctsG, int(nb_train/50)))
plt.plot(running_mean(corrects, int(nb_train/50)))

In [ ]:
nb_test = 1000
correctsG_test =[]
labelsG_test = []
for k in range(nb_test):
    x,l = generator.generate_input(len_r=seq_max_len,true_parent=True)
    y = RNNG(x)
    _,preds = torch.max(y.data,1)
    correctsG_test.append(preds.item() == l.data.item())
    labelsG_test.append(l.data)

In [ ]:
np.sum(correctsG_test)/nb_test

In [ ]:
nb_test = 1000
correctsG_test =[]
labelsG_test = []
for k in range(nb_test):
    x,l = generator.generate_input(len_r=seq_max_len, hard_false = True)
    y = RNNG(x)
    _,preds = torch.max(y.data,1)
    correctsG_test.append(preds.item() == l.data.item())
    labelsG_test.append(l.data)

In [ ]:
np.sum(correctsG_test)/nb_test

In [ ]:
nb_test = 1000
correctshG_test =[]
labelshG_test = []
for k in range(nb_test):
    x,l = generator.generate_input_hard()
    y = RNNG(x)
    _,preds = torch.max(y.data,1)
    correctshG_test.append(preds.item() == l.data.item())
    labelshG_test.append(l.data)

In [ ]:
np.sum(correctshG_test)/nb_test

## LSTM

In [ ]:
class LSTMNet(nn.Module):
    def __init__(self, dim_input=10, dim_recurrent=50, num_layers=4, dim_output=2):
        super(LSTMNet, self).__init__()
        self.lstm = nn.LSTM(input_size = dim_input,
                           hidden_size = dim_recurrent,
                           num_layers = num_layers)
        self.fc_o2y = nn.Linear(dim_recurrent,dim_output)

    def forward(self, x):
        x = x.unsqueeze(1)
        output, _ = self.lstm(x)
        output = output.squeeze(1)
        output = output.narrow(0, output.size(0)-1,1)
        return self.fc_o2y(F.relu(output))

lstm = gpu(LSTMNet(dim_input = nb_symbol))

In [ ]:
x, l = generator.generate_input()

In [ ]:
lstm(x)

In [ ]:
optimizerL = torch.optim.Adam(lstm.parameters(),lr=1e-3)
loss_tL = []
correctsL =[]
labelsL = []
start = time.time()
for k in range(nb_train):
    x,l = generator.generate_input(hard_false = False)
    y = lstm(x)
    loss = cross_entropy(y,l)
    _,preds = torch.max(y.data,1)
    correctsL.append(preds.item() == l.data.item())
    optimizerL.zero_grad()
    loss.backward()
    optimizerL.step()
    loss_tL.append(loss)
    labelsL.append(l.item())
print(time.time() - start)

In [ ]:
plt.plot(running_mean(loss_tL,int(nb_train/50)))
plt.plot(running_mean(loss_tG,int(nb_train/50)))
plt.plot(running_mean(loss_t,int(nb_train/50)))

In [ ]:
plt.plot(running_mean(correctsL,int(nb_train/50)))
plt.plot(running_mean(correctsG,int(nb_train/50)))
plt.plot(running_mean(corrects,int(nb_train/50)))

In [ ]:
nb_test = 1000
correctsL_test =[]
labelsL_test = []
for k in range(nb_test):
    x,l = generator.generate_input(len_r=seq_max_len, true_parent=True)
    y = lstm(x)
    _,preds = torch.max(y.data, 1)
    correctsL_test.append(preds.item() == l.data.item())
    labelsL_test.append(l.data)

In [ ]:
np.sum(correctsL_test)/nb_test

In [ ]:
nb_test = 1000
correctsL_test =[]
labelsL_test = []
for k in range(nb_test):
    x,l = generator.generate_input(len_r=seq_max_len,true_parent=False,hard_false = True)
    y = lstm(x)
    _,preds = torch.max(y.data,1)
    correctsL_test.append(preds.item() == l.data.item())
    labelsL_test.append(l.data)

In [ ]:
np.sum(correctsL_test)/nb_test

In [ ]:
nb_test = 1000
correctshL_test =[]
labelshL_test = []
for k in range(nb_test):
    x,l = generator.generate_input_hard()
    y = lstm(x)
    _,preds = torch.max(y.data,1)
    correctshL_test.append(preds.item() == l.data.item())
    labelshL_test.append(l.data)

In [ ]:
np.sum(correctshL_test)/nb_test

## Pred Engine Failure RNN

In [ ]:
%mkdir data
%cd data
!wget 'https://www.di.ens.fr/~lelarge/CMAPSSData.zip'
!unzip CMAPSSData.zip
%cd ..

/content/data/data
--2024-05-21 23:07:07--  https://www.di.ens.fr/~lelarge/CMAPSSData.zip
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘CMAPSSData.zip’

CMAPSSData.zip          [        <=>         ]  11.85M  6.62MB/s    in 1.8s    

2024-05-21 23:07:09 (6.62 MB/s) - ‘CMAPSSData.zip’ saved [12425978]

Archive:  CMAPSSData.zip
  inflating: Damage Propagation Modeling.pdf  
  inflating: readme.txt              
  inflating: RUL_FD001.txt           
  inflating: RUL_FD002.txt           
  inflating: RUL_FD003.txt           
  inflating: RUL_FD004.txt           
  inflating: test_FD001.txt          
  inflating: test_FD002.txt          
  inflating: test_FD003.txt          
  inflating: test_FD004.txt          
  inflating: train_FD001.txt         
  inflating: train_FD002.txt         
  inflating: train_FD0

In [ ]:
def get_CMAPSSData(nb_file):
    dataset_train = pd.read_csv('./data/train_FD00{}.txt'.format(nb_file),
                                sep=' ', header=None).drop([26, 27], axis=1)
    dataset_test = pd.read_csv('./data/test_FD00{}.txt'.format(nb_file),
                               sep=' ', header=None).drop([26, 27], axis=1)
    test_truth = pd.read_csv('./data/RUL_FD00{}.txt'.format(nb_file),
                             sep=' ', header=None).drop([1], axis=1)
    col_names = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8',
                 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']
    dataset_train.columns = col_names
    dataset_test.columns = col_names
    test_truth.columns = ['more']
    test_truth['id'] = test_truth.index + 1
    rul = pd.DataFrame(dataset_test.groupby('id')['cycle'].max()).reset_index()
    rul.columns = ['id', 'max']
    test_truth['rtf'] = test_truth['more'] + rul['max']
    test_truth.drop('more', axis=1, inplace=True)
    dataset_test = dataset_test.merge(test_truth, on=['id'], how='left')
    dataset_test['ttf'] = dataset_test['rtf'] - dataset_test['cycle']
    dataset_test.drop('rtf', axis=1, inplace=True)
    dataset_train['ttf'] = dataset_train.groupby(['id'])['cycle'].transform(max) - dataset_train['cycle']
    features_col_name = ['setting1', 'setting2', 'setting3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8',
                         's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']
    target_col_name = 'ttf'
    relevant_features_col_name = []
    for col in features_col_name:
        if not (len(dataset_train[col].unique()) == 1):
            relevant_features_col_name.append(col)
    sc = MinMaxScaler()
    dataset_train[features_col_name] = sc.fit_transform(dataset_train[features_col_name])
    dataset_test[features_col_name] = sc.transform(dataset_test[features_col_name])
    return dataset_train, dataset_test, relevant_features_col_name, target_col_name

def to_lists_of_tensors(dataset, features_col_name, target_col_name):
    X, y = [], []
    nb_sequences = max(dataset['id'])
    for i in range(1, nb_sequences + 1):
        df_zeros = dataset.loc[dataset['id'] == i]
        df_one_x = df_zeros[features_col_name]
        df_one_y = df_zeros[target_col_name]
        X.append(torch.from_numpy(np.expand_dims(df_one_x.values, 1)).type(torch.FloatTensor))
        y.append(torch.from_numpy(df_one_y.values).type(torch.FloatTensor))
    return X, y

def convert_train_and_test_to_appropriate_format(dataset_train, dataset_test, features_col_name, target_col_name):
    X_train, y_train = to_lists_of_tensors(dataset_train, features_col_name, target_col_name)
    X_test, y_test = to_lists_of_tensors(dataset_test, features_col_name, target_col_name)
    return X_train, y_train, X_test, y_test

 CMAPSSData.zip			    RUL_FD001.txt   test_FD001.txt   train_FD001.txt
'Damage Propagation Modeling.pdf'   RUL_FD002.txt   test_FD002.txt   train_FD002.txt
 data				    RUL_FD003.txt   test_FD003.txt   train_FD003.txt
 readme.txt			    RUL_FD004.txt   test_FD004.txt   train_FD004.txt
 CMAPSSData.zip			    RUL_FD002.txt    test_FD002.txt    train_FD002.txt
'Damage Propagation Modeling.pdf'   RUL_FD003.txt    test_FD003.txt    train_FD003.txt
 readme.txt			    RUL_FD004.txt    test_FD004.txt    train_FD004.txt
 RUL_FD001.txt			    test_FD001.txt   train_FD001.txt


In [ ]:
%pycat ./data/readme.txt

In [ ]:
dataset_train, dataset_test, features_col_name, target_col_name = get_CMAPSSData(1)
X_train, y_train, X_test, y_test = convert_train_and_test_to_appropriate_format(dataset_train, dataset_test, features_col_name, target_col_name)

In [ ]:
dataset_train.head()

In [ ]:
X_train[0].shape

In [ ]:
class GRUnet(nn.Module):
    def __init__(self, dim_input, num_layers, dim_hidden, dim_output=2):
        super(GRUnet, self).__init__()
        self.gru = nn.GRU(input_size = dim_input,
                          hidden_size = dim_hidden, num_layers = num_layers)
        self.hidden_to_two = nn.Linear(dim_hidden, dim_output)

    def forward(self, x):
        gru_output, _ = self.gru(x)
        gru_output = gru_output.squeeze(1)
        two_output = self.hidden_to_two(gru_output)
        return torch.exp(two_output)

In [ ]:
model = GRUnet(dim_input=len(features_col_name), num_layers=3, dim_hidden=50)
model = model.to(device)

In [ ]:
output = model(X_train[0].to(device))

In [ ]:
output.shape

In [ ]:
class weibull_loss(nn.Module):
    def __init__(self):
        super(weibull_loss, self).__init__()
        self.epsilon = 1e-6

    def forward(self, output, y):
        ya = (y + self.epsilon) / (output[:, 0])
        beta = output[:, 1]
        likelihoods = torch.log(beta) + beta * torch.log(ya) - torch.log(y+self.epsilon)- ya ** beta
        return -likelihoods.mean()

In [ ]:
loss_fn = weibull_loss()
loss_fn(output.squeeze(), y_train[0].to(device))

In [ ]:
def train_epoch(X_train, y_train, model, loss_fn, optimizer, device):
    nb_train_sequences = len(y_train)
    model.train()
    losses = []
    for k in range(nb_train_sequences):
        X, y = X_train[k], y_train[k]
        pred = model(X.to(device))
        loss = loss_fn(pred.squeeze(), y.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return np.mean(losses)

def test_epoch(X_test, y_test, model, loss_fn, device):
    nb_test_sequences = len(y_test)
    model.eval()
    losses = []
    for k in range(nb_test_sequences):
        X, y = X_test[k], y_test[k]
        pred = model(X.to(device))
        loss = loss_fn(pred.squeeze(), y.to(device))
        losses.append(loss.item())
    return np.mean(losses)

def fit(model, X_train, y_train, X_test, y_test, optimizer, loss_fn, nb_epochs, device):
    train_loss_t = []
    test_loss_t = []
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=1, verbose='True',
                                                           threshold=0.001)
    for epoch in range(0, nb_epochs):
        message1 = 'Epoch: {}/{}'.format(epoch + 1, nb_epochs)
        print(message1)
        train_loss = train_epoch(X_train, y_train, model, loss_fn, optimizer,device)
        test_loss = test_epoch(X_test, y_test, model, loss_fn,device)
        message2 = 'Train set: Average loss: {:.4f}\n'.format(train_loss)
        message3 = 'Test set: Average loss: {:.4f} - Learning rate : {:.4f}\n'.format(test_loss,optimizer.param_groups[0]['lr'])
        scheduler.step(test_loss)
        train_loss_t.append(train_loss)
        test_loss_t.append(test_loss)
        print(message2)
        print(message3)
    return model, train_loss_t, test_loss_t

In [ ]:
model = GRUnet(dim_input=len(features_col_name), num_layers=3, dim_hidden=50, dim_output=2)
model = model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = weibull_loss()
nb_epochs = 50

In [ ]:
model, train_loss_t, test_loss_t = fit(model, X_train, y_train, X_test, y_test, optimizer, loss_fn, nb_epochs, device)

In [ ]:
def plot_losses(train_loss_t, test_loss_t):
    nb_epochs = len(train_loss_t)
    plt.plot(range(nb_epochs), train_loss_t, color='orange', label='Loss on the training set')
    plt.plot(range(nb_epochs), test_loss_t, color='green', label='Loss on the testing set')
    plt.legend()
    plt.show()

In [ ]:
plot_losses(train_loss_t, test_loss_t)

In [ ]:
max_val = np.zeros(len(y_train))
for i,y in enumerate(y_train):
    max_val[i] = y[0].item()
baseline = np.mean(max_val)

In [ ]:
def compute_np(model, X_test, y_test, baseline=baseline, device=device, max_size=303):
    n_test = len(X_test)
    all_pred = np.empty((n_test,max_size,2))
    all_y = np.empty((n_test,max_size))
    base_pred = np.empty((n_test,max_size))
    all_pred[:] = np.NaN
    all_y[:] = np.NaN
    base_pred[:] = np.NaN
    list_npred = []
    for k in range(n_test):
        pred = model(X_test[k].to(device))
        pred_np = pred.cpu().detach().numpy()
        n_pred = pred_np.shape[0]
        list_npred.append(n_pred)
        all_pred[k,:n_pred,:] = pred_np
        all_y[k,:n_pred] = y_test[k].numpy()
        base_pred[k,:n_pred] = baseline - range(n_pred)
    return all_pred, all_y, base_pred, list_npred

In [ ]:
all_pred, all_y, base_pred, list_npred = compute_np(model, X_test, y_test)
pred_fail = all_pred[:,:,0]*gamma(1+1/all_pred[:,:,1])

In [ ]:
k = 50
plt.plot(pred_fail[k,:], label='predicted')
plt.plot(all_y[k], label='true')
plt.plot(base_pred[k], label='baseline')
plt.legend()

In [ ]:
def RMSE(pred_fail, all_y):
    return np.sqrt((pred_fail-all_y)**2)

In [ ]:
res= RMSE(pred_fail,all_y)
res_base = RMSE(base_pred, all_y)

In [ ]:
plt.plot(res[k])
plt.plot(res_base[k])

In [ ]:
plt.plot(np.nanmean(res,0), label='RMSE')
plt.plot(np.nanmean(res_base,0), label='RMSE baseline')
plt.legend()

In [ ]:
np.nanmean(res), np.nanmean(res_base)

In [ ]:
last_indices = list((~np.isnan(res)).sum(axis = 1) - 1)

In [ ]:
np.mean([res[i,j] for i,j in enumerate(last_indices)])

In [ ]:
np.mean([res_base[i,j] for i,j in enumerate(last_indices)])

In [ ]:
plot = sns.jointplot(x=[all_y[i,j] for i,j in enumerate(last_indices)],
                     y=[base_pred[i,j] for i,j in enumerate(last_indices)],
                     dropna=True, kind="kde", n_levels=30, color="g")
plot.ax_joint.plot([0,150], [0,150], 'b-', linewidth = 2)
plot.set_axis_labels('true', 'predicted')

In [ ]:
plot = sns.jointplot(x=[all_y[i,j] for i,j in enumerate(last_indices)],
                     y=[pred_fail[i,j] for i,j in enumerate(last_indices)],
                     dropna=True, kind="kde", n_levels=30, color="g")
plot.ax_joint.plot([0,150], [0,150], 'b-', linewidth = 2)
plot.set_axis_labels('true', 'predicted')